In [1]:
from src.component.data_info import *

In [ ]:
def fetch_ride_events_from_data_warehouse(
    from_date: datetime,
    to_date: datetime
) -> pd.DataFrame:
    """
    This function is used to simulate production data by sampling historical data
    from 52 weeks ago (i.e., 1 year).
    It has been adjusted for daily data files instead of monthly data files.

    Args:
        from_date (datetime): Start date of the target range.
        to_date (datetime): End date of the target range (exclusive).
    
    Returns:
        pd.DataFrame: Simulated production data with shifted dates.
    """
    # Calculate the adjusted historical dates
    from_date_ = from_date - timedelta(days=7 * 52)
    to_date_ = to_date - timedelta(days=7 * 52)

    print(f'Fetching demand events from {from_date_} to {to_date_} (historical data)')

    all_data = []  # List to store daily DataFrames

    # Loop through each day in the historical range
    for single_date in pd.date_range(start=from_date_, end=to_date_ - timedelta(days=1)):
        # Fetch the daily data file
        daily_data = load_full_data(
            start_date=single_date.strftime('%Y-%m-%d'),
            end_date=single_date.strftime('%Y-%m-%d')
        )
        if not daily_data.empty:
            all_data.append(daily_data)

    # Concatenate all daily DataFrames
    if all_data:
        demand_values = pd.concat(all_data, ignore_index=True)
    else:
        print("No historical data found.")
        return pd.DataFrame()

    # Convert 'date' column to datetime for filtering and shifting
    demand_values['date'] = pd.to_datetime(demand_values['date'])

    # Filter the data to ensure it matches the exact historical range
    demand_values = demand_values[(demand_values['date'] >= from_date_) & (demand_values['date'] < to_date_)]

    # Shift the 'date' column forward by 52 weeks to simulate production data
    demand_values['date'] += timedelta(days=7 * 52)

    # Sort the data by sub_region_code and date
    demand_values.sort_values(by=['sub_region_code', 'date'], inplace=True)

    return demand_values


In [ ]:
electricity_demand_data=